# Use LLM for metadata extraction from a query

This demo uses a large language model for metadata extraction from a query.

Uses OpenAI model

In [ ]:
!pip3 install openai==1.39.0 pydantic==2.8.2

In [2]:
from ipython_secrets import get_secret
import os

secret = get_secret('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = secret


In [3]:
from openai import OpenAI
from pydantic import BaseModel
import json
from typing import Literal, Optional

# Create client to call model
api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(
    api_key=api_key,
)

# Create classifier
class ContentTopic(BaseModel):
    topic: Optional[Literal[
        "nutritional_information",
        "equipment",
        "cooking_technique",
        "recipe"
    ]]

function_definition = {
    "name": "classify_topic",
    "description": "Extract the key topics from the query",
    "parameters": json.loads(ContentTopic.schema_json())
}
# The topic classifier uses few-shot examples to optimize the classification task.
def get_topic(query: str):
    response = client.chat.completions.create(
    model="gpt-4o-mini", 
    functions=[function_definition], 
    function_call={ "name": function_definition["name"] },
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": """Extract the topic of the following user query about cooking.
Only use the topics present in the content topic classifier function.
If you cannot tell the query topic or it is not about cooking, respond `null`. Output JSON.
You MUST choose one of the given content topic types.
Example 1:
User:  "How many grams of sugar are in a banana?"
Assistant: '{"topic": "nutritional_information"}'
Example 2:
User: "What are the ingredients for a classic margarita?"
Assistant: '{"topic": "recipe"}'
Example 3:
User: "What kind of knife is best for chopping vegetables?"
Assistant: '{"topic": "equipment"}'
Example 4:
User: "What is a quick recipe for chicken stir-fry?"
Assistant: '{"topic": "recipe"}'
Example 5:
User: Who is the best soccer player ever?
Assistant: '{"topic": null}'
Example 6:
User: Explain gravity to me
Assistant: '{"topic": null}'""",
          },
          {
              "role": "user",
              "content": query
          }
      ],
    )
    content = ContentTopic.model_validate(json.loads(response.choices[0].message.function_call.arguments)) 
    return content.topic

## Test the classifier
queries = [
    "what's a recipe for vegetarian spaghetti?",
    "what is the best way to poach an egg?",
    "What blender setting should I use to make a fruit smoothie?",
    "Can you give me a recipe for chocolate chip cookies?",
    "Why is the sky blue?"
]
for query in queries:
    print(f"Query: {query}")
    print(f"Topic: {get_topic(query)}")
    print("---")

    

Query: what's a recipe for vegetarian spaghetti?
Topic: recipe
---
Query: what is the best way to poach an egg?
Topic: cooking_technique
---
Query: What blender setting should I use to make a fruit smoothie?
Topic: equipment
---
Query: Can you give me a recipe for chocolate chip cookies?
Topic: recipe
---
Query: Why is the sky blue?
Topic: None
---
